In [42]:
!pip install paho-mqtt tensorflow

In [43]:
import paho.mqtt.client as mqtt
import tensorflow.lite as tflite
import numpy as np
import json

In [44]:
# --- Charger le modèle TFLite ---
interpreter = tflite.Interpreter(model_path="/content/drive/MyDrive/Colab Notebooks/anomaly_detector.tflite")
interpreter.allocate_tensors()

In [45]:
# Récupération des indices des tensors
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

In [46]:
# --- Configuration du MQTT ---
BROKER = "broker.hivemq.com"
PORT = 1883
TOPIC_SUB = "esp32/data"
TOPIC_PUB = "esp32/indicator"

In [47]:
# Définition des seuils pour détecter les anomalies
mq2_threshold = 0.37
mq135_threshold = 0.67

In [48]:
def on_connect(client, userdata, flags, rc):
    print("Connecté au broker MQTT !")
    client.subscribe(TOPIC_SUB)

In [49]:

def on_message(client, userdata, msg):
    print(f"Message reçu : {msg.payload.decode()}")
    try:
        data = json.loads(msg.payload.decode())
        # Normalisation des données avec les minmax de l'échantillon d'entrainement
        temp = (data["temperature"] - (-4.9)) / (59.6 - (-4.9))
        hum = (data["humidity"] - 36) / (95 - 36)
        actual_mq2 = (data["mq2_gas"] - 873) / (5588 - 873)
        actual_mq135 = (data["mq135_air_quality"] - 0) / (65416 - 0)

        # Mise en forme pour le modèle (TensorFlow Lite attend un array numpy)
        input_data = np.array([[temp, hum]], dtype=np.float32)
        interpreter.set_tensor(input_details[0]['index'], input_data)
        interpreter.invoke()
        output_data = interpreter.get_tensor(output_details[0]['index'])

        predicted_mq2, predicted_mq135 = output_data[0]

        # Calcul des résidus
        mq2_residual = abs(actual_mq2 - predicted_mq2)
        mq135_residual = abs(actual_mq135 - predicted_mq135)

         # Détermination de l'indicateur
        if mq2_residual > mq2_threshold and mq135_residual > mq135_threshold:
            indicator = 3
        elif mq2_residual > mq2_threshold:
            indicator = 1
        elif mq135_residual > mq135_threshold:
            indicator = 2
        else:
            indicator = 0

        print(f"Résultat de l'inférence : MQ2 = {predicted_mq2:.2f}, MQ135 = {predicted_mq135:.2f}")
        print(f"Résidus : MQ2 = {mq2_residual:.2f}, MQ135 = {mq135_residual:.2f}")
        print(f"Indicateur : {indicator}")

        # Publier l'indicateur sur MQTT
        client.publish(TOPIC_PUB, json.dumps({"indicator": indicator}))

    except Exception as e:
        print(f"Erreur : {e}")

In [50]:
client = mqtt.Client()
client.on_connect = on_connect
client.on_message = on_message

client.connect(BROKER, PORT, 60)
client.loop_forever()

<ipython-input-50-625fa7f42307>:1: DeprecationWarning: Callback API version 1 is deprecated, update to latest version
  client = mqtt.Client()


Connecté au broker MQTT !
Message reçu : {"temperature":23.96,"humidity":48.10,"mq2_gas":4168,"mq135_air_quality":1458}
Résultat de l'inférence : MQ2 = 0.40, MQ135 = 0.05
Résidus : MQ2 = 0.30, MQ135 = 0.03
Indicateur : 0
Message reçu : {"temperature":23.20,"humidity":49.60,"mq2_gas":4868,"mq135_air_quality":418}
Résultat de l'inférence : MQ2 = 0.41, MQ135 = 0.05
Résidus : MQ2 = 0.44, MQ135 = 0.05
Indicateur : 1
Message reçu : {"temperature":21.52,"humidity":40.45,"mq2_gas":3508,"mq135_air_quality":298}
Résultat de l'inférence : MQ2 = 0.38, MQ135 = 0.06
Résidus : MQ2 = 0.18, MQ135 = 0.05
Indicateur : 0
Message reçu : {"temperature":21.28,"humidity":42.70,"mq2_gas":3328,"mq135_air_quality":998}
Résultat de l'inférence : MQ2 = 0.39, MQ135 = 0.06
Résidus : MQ2 = 0.13, MQ135 = 0.04
Indicateur : 0
Message reçu : {"temperature":26.00,"humidity":50.50,"mq2_gas":5388,"mq135_air_quality":658}
Résultat de l'inférence : MQ2 = 0.40, MQ135 = 0.05
Résidus : MQ2 = 0.55, MQ135 = 0.04
Indicateur : 1
Mes

KeyboardInterrupt: 